In [2]:
import collections
import numpy as np
import torch
import torch.nn.functional as F
from random import shuffle
import random
import pandas as pd
from sklearn.metrics import f1_score
from sklearn.preprocessing import MultiLabelBinarizer
from nltk import ngrams
from torchtext.data.functional import generate_sp_model, sentencepiece_tokenizer, load_sp_model
from torchtext.vocab import build_vocab_from_iterator
import csv

In [2]:
device = torch.device('cuda:0')
# device = torch.device('cpu')

In [ ]:
# Open the file 'words_with_hidden_train.txt' in write mode.
with open ("words_with_hidden_train.txt", "w") as f1:
    # Initialize an empty list 'word_list' to store words.
    word_list = []

    # Open the file 'words_250000_train.txt' in read mode.
    with open ("words_250000_train.txt", 'r') as f2:
        # Read all lines from the file into a list 'Lines'.
        Lines = f2.readlines()
        
        # Iterate over each line (word) in the list 'Lines'.
        for word in Lines:
            # Convert the word to a numpy array.
            word = np.array([*word])
            
            # Generate five different versions of the word with random masks.
            for perm in range(5):
                # Generate a random number 'flag'.
                flag = np.random.rand()

                # Set different thresholds for the random mask based on the value of 'flag'.
                if flag < 0.33:
                    inp_mask = np.random.rand(*word.shape) < 0.1
                elif 0.1 <= flag < 0.33:
                    inp_mask = np.random.rand(*word.shape) < 0.2
                elif 0.33 <= flag < 0.66:
                    inp_mask = np.random.rand(*word.shape) < 0.5
                else:
                    inp_mask = np.random.rand(*word.shape) < 0.8

                # Get the index of elements where the mask is true.
                index_to_encode = inp_mask.nonzero()[0]

                # Create a copy of the original word and apply the mask.
                word_masked = np.copy(word)
                word_masked[index_to_encode] = '_'
                
                # Append the masked word to the list 'word_list'.
                word_list += [''.join(word_masked)]
            
            # After generating 5 masked versions, append the original word and a fully masked word to 'word_list'.
            word_list += [''.join(word)]
            word_list += ['_'*len(word)]
    
    # Add a newline character to the end of each word in 'word_list'.
    word_list = [word + '\n' for word in word_list]
    
    # Write the words from 'word_list' into the file 'words_with_hidden_train.txt'.
    f1.writelines(word_list)

In [3]:
with open ("words_with_hidden_train.txt", "w") as f1:
    word_list = []
    with open ("words_250000_train.txt", 'r') as f2:
        Lines = f2.readlines()
        
        for word in Lines:
            word = np.array([*word])
            
            for perm in range(5):
                flag = np.random.rand()
                if flag < 0.33:
                    inp_mask = np.random.rand(*word.shape) < 0.1
                elif 0.1 <= flag < 0.33:
                    inp_mask = np.random.rand(*word.shape) < 0.2
                elif 0.33 <= flag < 0.66:
                    inp_mask = np.random.rand(*word.shape) < 0.5
                else:
                    inp_mask = np.random.rand(*word.shape) < 0.8

                index_to_encode = inp_mask.nonzero()[0]

                word_masked = np.copy(word)
                word_masked[index_to_encode] = '_'
                
                word_list += [''.join(word_masked)]
            word_list += [''.join(word)]
            word_list += ['_'*len(word)]
    word_list = [word + '\n' for word in word_list]
    f1.writelines(word_list)

In [3]:
generate_sp_model("words_with_hidden_train.txt", vocab_size=16384, model_type='bpe', model_prefix='bpe_words_16384')

sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=words_with_hidden_train.txt --model_prefix=bpe_words_16384 --vocab_size=16384 --model_type=bpe
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: words_with_hidden_train.txt
  input_format: 
  model_prefix: bpe_words_16384
  model_type: BPE
  vocab_size: 16384
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 0
  bos_id: 1
  eos_id: 2
  pad_id: -1
  unk_piece: <unk>
  bos_piece: <s>
  eos_piec

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=409 size=1160 all=133091 active=11344 piece=tit
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=400 size=1180 all=134939 active=13192 piece=▁tric
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=391 size=1200 all=136569 active=14822 piece=▁hol
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=390 min_freq=58
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=380 size=1220 all=138050 active=8241 piece=lav
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=372 size=1240 all=139731 active=9922 piece=kn
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=367 size=1260 all=141458 active=11649 piece=▁gar
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=360 size=1280 all=143230 active=13421 piece=▁phy
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=354 size=1300 all=144886 active=15077 piece=thy
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=354 min_freq=53
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=347 size=1

bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=135 min_freq=25
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=134 size=2820 all=237802 active=12696 piece=ranchi
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=132 size=2840 all=238634 active=13528 piece=eted
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=131 size=2860 all=239548 active=14442 piece=scri
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=130 size=2880 all=240267 active=15161 piece=asia
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=129 size=2900 all=241262 active=16156 piece=idin
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=129 min_freq=25
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=128 size=2920 all=242133 active=12870 piece=boat
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=127 size=2940 all=243057 active=13794 piece=pli
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=126 size=2960 all=243860 active=14597 piece=ways
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=125

bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=79 min_freq=18
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=79 size=4320 all=293363 active=15184 piece=▁school
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=78 size=4340 all=293949 active=15770 piece=etes
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=78 size=4360 all=294589 active=16410 piece=ophile
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=77 size=4380 all=295184 active=17005 piece=ules
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=77 size=4400 all=295781 active=17602 piece=▁wash
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=77 min_freq=17
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=76 size=4420 all=296393 active=15371 piece=tac
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=76 size=4440 all=297035 active=16013 piece=▁bud
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=76 size=4460 all=297556 active=16534 piece=▁cereb
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=75 size

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=55 size=5800 all=330680 active=18489 piece=▁mind
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=55 min_freq=14
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=55 size=5820 all=331186 active=17019 piece=▁predis
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=54 size=5840 all=331604 active=17437 piece=bili
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=54 size=5860 all=332094 active=17927 piece=stat
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=54 size=5880 all=332489 active=18322 piece=lytic
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=54 size=5900 all=332893 active=18726 piece=isting
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=54 min_freq=14
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=53 size=5920 all=333301 active=17030 piece=gis
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=53 size=5940 all=333692 active=17421 piece=onin
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=53 size=

...

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=32 size=8960 all=385151 active=19935 piece=count
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=32 size=8980 all=385457 active=20241 piece=▁arra
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=32 size=9000 all=385678 active=20462 piece=▁isob
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=32 min_freq=10
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=32 size=9020 all=385916 active=19510 piece=▁unbo
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=32 size=9040 all=386213 active=19807 piece=▁halff
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=32 size=9060 all=386483 active=20077 piece=ateship
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=32 size=9080 all=386778 active=20372 piece=▁palaeo
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=31 size=9100 all=387090 active=20684 piece=bp
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=31 min_freq=10
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=31 s

bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27 size=10480 all=403780 active=21068 piece=▁secre
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27 size=10500 all=404002 active=21290 piece=olithic
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=27 min_freq=9
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27 size=10520 all=404266 active=20445 piece=▁turbin
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=27 size=10540 all=404536 active=20715 piece=ophyllous
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=10560 all=404747 active=20926 piece=jah
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=10580 all=404899 active=21078 piece=arks
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=10600 all=405145 active=21324 piece=imon
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=26 min_freq=9
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=26 size=10620 all=405409 active=20503 piece=rnal
bpe_model_trainer.cc(258) LOG(INFO) Added: f

bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=23 min_freq=8
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=23 size=11920 all=419151 active=21098 piece=beaten
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=23 size=11940 all=419441 active=21388 piece=otyled
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=23 size=11960 all=419649 active=21596 piece=▁chlam
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=23 size=11980 all=419845 active=21792 piece=▁phall
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=23 size=12000 all=420035 active=21982 piece=bergite
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=23 min_freq=8
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=23 size=12020 all=420259 active=21216 piece=▁foresh
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=23 size=12040 all=420469 active=21426 piece=oprotein
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=23 size=12060 all=420713 active=21670 piece=encephalic
bpe_model_trainer.cc(258) LOG(IN

bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=19 min_freq=8
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 size=13620 all=434442 active=21881 piece=ensa
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 size=13640 all=434600 active=22039 piece=inum
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 size=13660 all=434779 active=22218 piece=onem
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 size=13680 all=434968 active=22407 piece=smic
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 size=13700 all=435094 active=22533 piece=▁evi
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=19 min_freq=8
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 size=13720 all=435198 active=21857 piece=▁vog
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 size=13740 all=435391 active=22050 piece=edral
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 size=13760 all=435549 active=22208 piece=liver
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=19 siz

bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=17 min_freq=7
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=15320 all=447409 active=22493 piece=▁krist
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=15340 all=447543 active=22627 piece=▁redis
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=15360 all=447686 active=22770 piece=▁thuri
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=15380 all=447831 active=22915 piece=▁yello
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=15400 all=447992 active=23076 piece=omerism
bpe_model_trainer.cc(167) LOG(INFO) Updating active symbols. max_freq=17 min_freq=7
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=15420 all=448157 active=22557 piece=turally
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=15440 all=448312 active=22712 piece=▁incred
bpe_model_trainer.cc(258) LOG(INFO) Added: freq=17 size=15460 all=448465 active=22865 piece=▁prerec
bpe_model_trainer.cc(258) LOG(INFO) 

In [4]:
4096*4

16384

In [4]:
def read_file(file):
    with open(file, encoding='utf-8') as f:
        reader = csv.reader(f, delimiter = '\n')
        yield from reader
        
def yield_token(file,spm):
    for src in read_file(file):
        if len(src)>0:
            yield from spm([src[0]])


spm_model_path = 'bpe_words_16384.model'
spm = sentencepiece_tokenizer(load_sp_model(spm_model_path))

vocab = build_vocab_from_iterator(yield_token('words_with_hidden_train.txt',spm),
                            specials=["<unk>", "<pad>", "<mask>", "_"])

padding_value = vocab(["<pad>"])[0]
vocab.set_default_index(vocab['<unk>'])

In [5]:
def encode(word, spm, vocab):
    """
    Converts a word into a list of tokenized segments.

    Args:
        word (str): The word to be tokenized.
        spm (SentencePieceProcessor): The tokenizer object.
        vocab (Vocab): The vocabulary object containing word-to-index mappings.

    Returns:
        list: A list of tokenized segments.
    """
    return vocab(list(spm([word]))[0])

def one_hot(n_classes, targets):
    """
    Converts a list of integers into a one-hot encoded matrix.

    Args:
        n_classes (int): The total number of classes.
        targets (list): The list of target integers to be one-hot encoded.

    Returns:
        np.ndarray: A one-hot encoded matrix of the target integers.
    """
    one_hot_targets = np.eye(n_classes)[targets]
    return one_hot_targets
  
def encode_row(row, vocab):
    """
    Encodes a row into a one-hot encoded vector.

    Args:
        row (list): The row to be one-hot encoded.
        vocab (Vocab): The vocabulary object containing character-to-index mappings.

    Returns:
        np.ndarray: A one-hot encoded vector of the row.
    """
    n_classes = len(vocab.chars)

    target = np.zeros((1,n_classes))
    for char_id in row:
        target += one_hot(n_classes, char_id)
    target = np.clip(target, 0, 1)
    return target

def get_ngram_masked_input_and_label(encoded_text, mask_token_id, vocab, max_len, spm, encoder2):    
    """
    Generates masked inputs and labels for masked language modeling.

    Args:
        encoded_text (list): The list of tokenized words.
        mask_token_id (int): The id of the mask token.
        vocab (Vocab): The vocabulary object containing character-to-index mappings.
        max_len (int): The maximum length of an encoded text.
        spm (SentencePieceProcessor): The tokenizer object.
        encoder2 (Encoder): The encoder object to convert characters to indices.

    Returns:
        np.ndarray: Masked input text.
        list: Labels for each position in the masked input.
        np.ndarray: Weights for each position in the masked input.
    """
    flag = np.random.rand() 
    encoded_text = np.array(encoded_text)
    

    # MASK AND PREDICT
    # Define the mask percentage based on the random flag.
    if flag < 0.32:
        inp_mask = np.random.rand(*encoded_text.shape) < 0.2
    elif 0.33 <= flag < 0.66:
        inp_mask = np.random.rand(*encoded_text.shape) < 0.5
    else:
        inp_mask = np.random.rand(*encoded_text.shape) < 0.8
    
    # Get the indices where the mask is true.
    index_to_encode = inp_mask.nonzero()[0]
    encoded_text_masked = np.copy(encoded_text)
    encoded_text_masked[index_to_encode] = mask_token_id
    

    # Set of unique masked characters.
    char_set = list(set(tuple(encoded_text[index_to_encode])))
    all_index = index_to_encode
    for char in char_set:
        all_index = np.concatenate([all_index, np.where(encoded_text_masked==char)[0]])


    # Create labels for each character in the text.
    y_labels = [encoder2.char_to_index[chr] for chr in encoded_text]
    

    # Replace all relevant positions in the text with mask token.
    encoded_text_masked[all_index] = mask_token_id
    sample_weights = np.zeros(encoded_text.shape) + 1e-5
    sample_weights[all_index] = 1
    
    # Returns the masked input, the corresponding labels, and weights for each position.
    return encoded_text_masked, y_labels, sample_weights

In [6]:
def collate_data_for_ngrams(data, encode, spm, vocab, max_len=16, enc2=None): 
    """
    Prepares and collates data for ngram modelling by masking words, encoding them and 
    padding them to a fixed length.

    Args:
        data (list): The list of words to be processed.
        encode (function): The function used to encode the words.
        spm (SentencePieceProcessor): The tokenizer object.
        vocab (Vocab): The vocabulary object containing word-to-index mappings.
        max_len (int, optional): The maximum length of the output tensors. Default is 16.
        enc2 (Encoder, optional): The encoder object to convert characters to indices.

    Returns:
        tuple: A tuple containing three tensors - 
               padded_word: The tensor of masked and encoded words, 
               padded_target: The tensor of target words,
               padded_weight: The tensor of weights for each character in the words.
    """

    # Initialize lists for the words, targets and weights.
    padded_word = []
    padded_target = []
    padded_weight = []

    # Iterate over each word in the provided data.
    for word in data:
        # Generate four different masked versions for each word.
        for permutation in range(4):
            # Get the masked version of the word, its corresponding labels, and the sample weights.
            word_masked, y_labels, sample_weights = get_ngram_masked_input_and_label(
              encoded_text=np.array([*word]), 
                mask_token_id=vocab(['_']), 
                vocab=vocab, 
                max_len=16,
                spm=spm,
                encoder2=enc2
            )

            # Encode the masked word.
            encoded_text_masked = encode(''.join(word_masked), spm, vocab)

            # Convert the masked word, labels, and weights to PyTorch tensors.
            x_masked_encoded = torch.tensor(list(encoded_text_masked[:max_len]), dtype=torch.int64)
            target_encoded = torch.tensor(list(y_labels[:max_len]), dtype=torch.float)
            sample_weights = torch.tensor(list(sample_weights[:max_len]), dtype=torch.float)

            # Determine the size of padding needed for the word and the labels.
            padding_size = max_len - x_masked_encoded.shape[0] if max_len - x_masked_encoded.shape[0] > 0 else 0
            padding_size2 = max_len - target_encoded.shape[0] if max_len - target_encoded.shape[0] > 0 else 0

            # Pad the word, labels, and weights to the maximum length.
            x_mask_padded = torch.cat([x_masked_encoded, torch.full((padding_size,), padding_value)], dim=0)
            target_padded = torch.cat([target_encoded, torch.full((padding_size2,), padding_value)], dim=0)
            sample_weights_padded = torch.cat([sample_weights, torch.full((padding_size2,), 0)], dim=0)

            # Append the padded word, labels, and weights to the respective lists.
            padded_word.append(x_mask_padded)
            padded_target.append(target_padded)
            padded_weight.append(sample_weights_padded)

    # Convert the lists of words, labels, and weights to PyTorch tensors.
    padded_word = torch.stack(padded_word, dim=0)
    padded_target = torch.stack(padded_target, dim=0).squeeze(1)
    padded_weight = torch.stack(padded_weight, dim=0)

    # Return the tensors of words, labels, and weights.
    return padded_word, padded_target, padded_weight


In [7]:
class Data:
    def __init__(self):
        self.full_dictionary = self.build_dictionary("words_250000_train.txt")
        self.full_dictionary_common_letter_sorted = collections.Counter("".join(self.full_dictionary)).most_common()

        self.chars = sorted(list(set(''.join(self.full_dictionary))))+ ['_'] + ['<pad>'] + ['bos'] + ['eos']
        self.char_to_index = dict((c, i) for i, c in enumerate(self.chars))
        self.index_to_char = dict((i, c) for i, c in enumerate(self.chars))
        
        self.weights = {}
        self.total_chars = 0
        for char in self.full_dictionary_common_letter_sorted:
            self.total_chars += char[1]
        
        for char in self.full_dictionary_common_letter_sorted:
            self.weights[char[0]] = self.total_chars/(26*char[1])

        
    def build_dictionary(self, dictionary_file_location):
        text_file = open(dictionary_file_location,"r")
        full_dictionary = text_file.read().splitlines()
        text_file.close()
        return full_dictionary

    
    def encode(self, word):
        return np.array([self.char_to_index[char] for char in word])

data_class = Data()

In [9]:
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm

shuffle(data_class.full_dictionary)

padded_word, padded_target, padded_weight = collate_data_for_ngrams(
    data = data_class.full_dictionary[0:10], 
    encode = encode, 
    spm = spm, 
    vocab = vocab,
    max_len = 16,
    enc2=data_class
)

In [68]:
from torch import nn
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 32):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        """
        Args:
            x: Tensor, shape [seq_len, batch_size, embedding_dim]
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

class TransformerNgramModel(torch.nn.Module):
    def __init__(self):
        super(TransformerNgramModel, self).__init__()
        d_model = 128
        vocab_size = 16384
        dropout = 0.1
        nhead = 2
        num_layers = 2
        dim_feedforward = 128
        max_len = 16
        
        self.emb = torch.nn.Embedding(vocab_size, d_model)
        
        self.pos_encoder = PositionalEncoding(
            d_model=d_model,
            dropout=dropout,
            max_len=max_len
        )

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout,
        )
        self.transformer_encoder = nn.TransformerEncoder(
            encoder_layer,
            num_layers=num_layers,
        )

        self.linear = torch.nn.Linear(d_model, 30)
        self.d_model = d_model
        
    def forward(self, x, src_key_padding_mask):
        x = self.emb(x) * math.sqrt(self.d_model)
        x = torch.permute(x, (1, 0, 2))
        x = self.pos_encoder(x)
        x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)
        x = self.linear(x).transpose(1,0).transpose(2,1)
        return x

In [62]:
padded_target = padded_target.to(dtype=torch.int64)

In [69]:
model=TransformerNgramModel().to(device)
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)
dataset=TensorDataset(padded_word, padded_target, padded_weight)

In [71]:
num_epochs = 51


train, val = torch.utils.data.random_split(dataset,
                                            [len(padded_word)-2048, 
                                            2048]
                                          )

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)

for epoch in range(num_epochs):
    
    train_loader = DataLoader(dataset, batch_size=128, shuffle=True)
    val_loader = DataLoader(dataset, batch_size=2048, shuffle=True)

    print('Epoch:', epoch)
    for index, sample_ in enumerate(train_loader):
        input_ = sample_[0].to(device)
        target = sample_[1].to(device)
        weight = sample_[2].to(device)
        

        logits = model(input_.to(device), src_key_padding_mask=(input_ == padding_value).to(device))

        loss = F.cross_entropy(logits, target, reduction='none')*weight
        loss = torch.sum(loss)/(torch.sum(weight) + 1e-10)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if index % 1000 == 0:
            print('Train Loss: ', loss.item(), 'index:', index)


    if epoch % 5 == 0:
        torch.save(model.state_dict(), './transformer_16384_epoch{}.pkl'.format(str(epoch)))

Epoch: 0
Train Loss:  2.774324893951416 index: 0
Train Loss:  2.74009108543396 index: 1000
Epoch: 1
Train Loss:  2.757533550262451 index: 0


KeyboardInterrupt: ignored